In [4]:
import numpy as np
import polars as pl
import time

In [2]:
blocks = pl.read_parquet('../out/train/building_blocks.parquet')
blocks.head()

index,smile,ecfp,ecfp_pca,onehot-pca,onehot_pca
i64,str,list[i64],list[f64],list[f64],list[f64]
0,"""Br.Br.NCC1CCCN…","[0, 0, … 0]","[-0.623631, 1.179455, … 0.042602]","[-4.098411, 0.498678, … -0.011183]","[-4.098411, 0.498678, … -0.011182]"
1,"""Br.NCc1cccc(Br…","[0, 0, … 0]","[-1.059422, 0.214484, … -0.026173]","[-3.521219, -3.102144, … 0.240914]","[-3.521219, -3.102144, … 0.240914]"
2,"""C#CCOc1ccc(CN)…","[0, 0, … 0]","[-0.7829, 0.581478, … 0.137127]","[-1.921474, 0.660365, … 0.950667]","[-1.921474, 0.660365, … 0.950667]"
3,"""C#CCOc1cccc(CN…","[0, 0, … 0]","[-0.613241, 0.365505, … 0.10522]","[-1.921474, 0.660365, … 0.950667]","[-1.921474, 0.660365, … 0.950667]"
4,"""C#CC[C@@H](CC(…","[0, 1, … 0]","[3.472075, 0.095289, … -0.086727]","[6.583925, 4.480118, … 0.27448]","[6.583925, 4.480118, … 0.274478]"


In [3]:
train = pl.read_parquet('../out/train/train/base/base-BRD4-01.parquet')
train.head()

id,protein_name,binds,buildingblock1_index,buildingblock2_index,buildingblock3_index
i64,str,i64,i64,i64,i64
1090779,"""BRD4""",0,5,3,0
1090782,"""BRD4""",0,5,3,1
1090788,"""BRD4""",0,5,3,3
1090794,"""BRD4""",0,5,3,11
1090800,"""BRD4""",0,5,3,14


In [ ]:
def features(dt, blocks):
    iblocks_ecfp_pca = [np.vstack(blocks['ecfp_pca'][dt[x]]) for x in ['buildingblock1_index', 'buildingblock2_index', 'buildingblock3_index']]
    iblocks_ecfp_pca = np.concatenate(iblocks_ecfp_pca, axis = 1)
    return(iblocks_ecfp_pca)

start_time = time.time()
x = features(train, blocks)
print(f"Elapsed time: {time.time() - start_time} seconds")

In [2]:
def features(dt, blocks):
    
    blocks_ecfp_pca = []
    for i in ['buildingblock1_index', 'buildingblock2_index', 'buildingblock3_index']:
        iblocks_ecfp_pca = blocks['ecfp_pca'][dt[i]].values
        iblocks_ecfp_pca = np.array([list(x) for x in iblocks_ecfp_pca]).astype('float')
        blocks_ecfp_pca.append(iblocks_ecfp_pca)
        del iblocks_ecfp_pca, i
        
    blocks_ecfp_pca = np.concatenate(blocks_ecfp_pca, axis = 1)
    return(blocks_ecfp_pca)

start_time = time.time()
x = features(train, blocks)
print(f"Elapsed time: {time.time() - start_time} seconds")

In [ ]:
def features(dt, blocks):

    blocks = blocks.select(['index', 'ecfp_pca'])
    dt = dt.select(['id', 'buildingblock1_index', 'buildingblock2_index', 'buildingblock3_index'])

    dt = dt.join(blocks, left_on = 'buildingblock1_index', right_on = 'index', how = 'inner').drop('buildingblock1_index').rename({'ecfp_pca': 'ecfp_pca1'})
    dt = dt.join(blocks, left_on = 'buildingblock2_index', right_on = 'index', how = 'inner').drop('buildingblock2_index').rename({'ecfp_pca': 'ecfp_pca2'})
    dt = dt.join(blocks, left_on = 'buildingblock3_index', right_on = 'index', how = 'inner').drop('buildingblock3_index').rename({'ecfp_pca': 'ecfp_pca3'})

    dt = dt.with_columns(
        pl.col('ecfp_pca1').list.concat(pl.col('ecfp_pca2').list.concat(pl.col('ecfp_pca3'))).cast(pl.Float32).alias('blocks_ecfp_pca')
    ).select(['id', 'blocks_ecfp_pca'])

    return(dt['blocks_ecfp_pca'])

start_time = time.time()
x = features(train, blocks)
print(f"Elapsed time: {time.time() - start_time} seconds")